In [1]:
import os
%pwd

'/Users/rithikreddy/Desktop/Text-Summarizer/research'

In [2]:
os.chdir("../") #changing the directory
%pwd

'/Users/rithikreddy/Desktop/Text-Summarizer'

In [3]:
#step3: entity updation
#entity is written type of a function
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_url: str
    local_data_file: Path
    unzip_dir: Path

In [4]:
from textsummarizer.constants import *
from textsummarizer.utils.common import read_yaml,create_directories

In [21]:
#step 4: updating the configuration manager.
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])
        
        data_ingestion_config = DataIngestionConfig(
            root_dir = config.root_dir,
            source_url=config.source_url,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )

        return data_ingestion_config

In [22]:
#step5: Update the Components
import os
import urllib.request as request    #to open and get the content of the url
import zipfile  #for unzip operation
from textsummarizer.logging import logger
from textsummarizer.utils.common import get_size

In [30]:
class DataIngestion:
    def __init__(self,config: DataIngestionConfig):
        self.config = config
    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_url,
                filename= self.config.local_data_file
            ) #retrives the url into the temporary location on disk, returns tuple
            logger.info(f"{filename} download! with the following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")

    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path,exist_ok=True)
        try:
            with zipfile.ZipFile(self.config.local_data_file,'r') as zip_ref:
                print("executing the zip file extraction")
                zip_ref.extractall(unzip_path)
        except zipfile.BadZipFile:
            logger.error("Failed to extract the zipfile:")

In [31]:
#step 6: creating and updating the pipeline
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    #data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-02-28 11:35:37,247: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-02-28 11:35:37,249: INFO: common: yaml file: params.yaml loaded successfully]
[2024-02-28 11:35:37,250: INFO: common: Successfully Created directory at: artifacts]
[2024-02-28 11:35:37,250: INFO: common: Successfully Created directory at: artifacts/data_ingestion]
[2024-02-28 11:35:38,220: INFO: 1840659251: artifacts/data_ingestion/data.zip download! with the following info: 
Connection: close
Content-Length: 23627009
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "2e7409b328d118a1d37018be788babf8bf9640386387da766ac100e96efa3b93"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 27F6:4580:CF5744:FD5F6E:65DF7C4F
Accept-Ranges: bytes
Date: Wed, 28 Feb 2024 19:35:37 GMT
Via: 1.1 varnish
X-Served-By: 